<a href="https://colab.research.google.com/github/aviraltyagi/DeepLearningArchitectures/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Input, Add, Dense, Flatten, ZeroPadding2D, BatchNormalization, Conv2D, AveragePooling2D, MaxPool2D, GlobalMaxPool2D, Activation
from tensorflow.keras.models import Model
from keras.initializers import glorot_uniform
import typing

In [ ]:
class CustomLayer(Layer):
  def __init__(self, filter, kernelSize, stride, padding, name, kernelInitializer, axis= 3):
    super(CustomLayer, self).__init__()
    self.Conv = Conv2D(filters= filter, kernel_size= kernelSize, strides= stride, padding= padding, name= name + "conv", kernel_initializer= kernelInitializer)
    self.BatchNorm = BatchNormalization(axis = axis, name= name + "bn")
    self.Activation = Activation('relu')
    #self.n = name

  def call(self, input, training = False, isActivation = True):
    x = self.Conv(input, training = training)
    x = self.BatchNorm(x)
    #print(self.n + "_conv")
    if isActivation:
      x = self.Activation(x)
      #print(self.n + "_active")
    
    return x

In [ ]:
class CustomBlock():
  def identity_Block(input, f, filters, stage, block):
    name_base = block + "S" + str(stage)

    F1, F2, F3 = filters

    x_shortcut = input

    # First Component of main path
    x = CustomLayer(filter= F1, kernelSize= (1,1), stride= (1,1), padding= 'valid', name= name_base + "_L1", kernelInitializer= glorot_uniform(seed= 0))(input, training = True)

    # Second Component of main path
    x = CustomLayer(filter= F2, kernelSize= (f,f), stride= (1,1), padding= 'same', name= name_base + "_L2", kernelInitializer= glorot_uniform(seed= 0))(x, training = True)

    # Third component of main path
    x = CustomLayer(filter= F3, kernelSize= (1,1), stride= (1,1), padding= 'valid', name= name_base + "_L3", kernelInitializer= glorot_uniform(seed= 0))(x, training = True, isActivation= False)

    # Final step: Add shortcut value to the main path
    x = Add()([x, x_shortcut])
    x = Activation('relu')(x, training= True)

    return x

  def convolutional_Block(input, f, filters, stage, block, s= 2):
    name_base = block + "S" + str(stage)

    F1, F2, F3 = filters

    x_shortcut = input

    # First Component of main path
    x = CustomLayer(filter= F1, kernelSize= (1,1), stride= (s,s), padding= 'valid', name= name_base + "_L1", kernelInitializer= glorot_uniform(seed= 0))(input, training = True)

    # Second Component of main path
    x = CustomLayer(filter= F2, kernelSize= (f,f), stride= (1,1), padding= 'same', name= name_base + "_L2", kernelInitializer= glorot_uniform(seed= 0))(x, training = True)

    # Third component of main path
    x = CustomLayer(filter= F3, kernelSize= (1,1), stride= (1,1), padding= 'valid', name= name_base + "_L3", kernelInitializer= glorot_uniform(seed= 0))(x, training = True, isActivation= False)

    # Shortcut path
    x_shortcut = Conv2D(filters= F3, kernel_size= (1,1), strides=(s,s), padding= 'valid', name= 'res' + name_base + '1', kernel_initializer= glorot_uniform(seed= 0))(x_shortcut)
    x_shortcut = BatchNormalization(axis= 3, name= 'bn' + name_base + '1')(x_shortcut)

    # Final step: Add shortcut value to the main path
    x = Add()([x, x_shortcut])
    x = Activation('relu')(x, training= True)

    return x    

In [ ]:
class CustomResNet(Model):
  def ResNet50(input_shape: typing.Tuple[int], classes: int) -> Model:
    x_input = Input(shape= input_shape)

    x = ZeroPadding2D(padding= (3,3))(x_input)

    # Stage 1
    x = CustomLayer(filter= 64, kernelSize= (7,7), stride= (2,2), padding= 'valid', name = 'S1_L1_', kernelInitializer= glorot_uniform(seed= 0))(x)
    x = MaxPool2D(pool_size= (3,3), strides= (2,2))(x)

    # Stage 2
    x = CustomBlock.convolutional_Block(input= x, f= 3, filters= [64, 64, 256], stage= 2, block= 'CB_', s= 1)
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [64, 64, 256], stage= 2, block= 'IB_1')
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [64, 64, 256], stage= 2, block= 'IB_2')

    # Stage 3
    x = CustomBlock.convolutional_Block(input= x, f= 3, filters= [128, 128, 512], stage= 3, block= 'CB_', s= 2)
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [128, 128, 512], stage= 3, block= 'IB_1')
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [128, 128, 512], stage= 3, block= 'IB_2')
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [128, 128, 512], stage= 3, block= 'IB_3')

    # Stage 4
    x = CustomBlock.convolutional_Block(input= x, f= 3, filters= [256, 256, 1024], stage= 4, block= 'CB_', s= 2)
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [256, 256, 1024], stage= 4, block= 'IB_1')
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [256, 256, 1024], stage= 4, block= 'IB_2')
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [256, 256, 1024], stage= 4, block= 'IB_3')
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [256, 256, 1024], stage= 4, block= 'IB_4')
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [256, 256, 1024], stage= 4, block= 'IB_5')

    # Stage 5
    x = CustomBlock.convolutional_Block(input= x, f= 3, filters= [512, 512, 2048], stage= 5, block= 'CB_', s= 2)
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [512, 512, 2048], stage= 5, block= 'IB_1')
    x = CustomBlock.identity_Block(input= x, f= 3, filters= [512, 512, 2048], stage= 5, block= 'IB_2')

    # Average Pool
    x = AveragePooling2D(pool_size= (2,2), name= 'avg_pool')(x)

    # Output layer
    x = Flatten()(x)
    output = Dense(units= classes, activation= 'softmax', name= 'output', kernel_initializer= glorot_uniform(seed= 0))(x)

    model = Model(inputs = x_input, outputs= output, name= 'ResNet50')

    return model

In [ ]:
model = CustomResNet.ResNet50(input_shape= (64, 64, 3), classes= 6)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
custom_layer (CustomLayer)      (None, 32, 32, 64)   9728        zero_padding2d[0][0]             
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 15, 15, 64)   0           custom_layer[0][0]               
___________________________________________________________________________________________